<a href="https://colab.research.google.com/github/g0dbot/COMP3610-GROUPM/blob/main/GroupM_M1_SEPv1-t1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
WIth the dataset including over 185,000 samples we greatly reduced the size of the training and testing samples while still maintaining a 1:1:1 ratio

In [1]:
! pip install -q kaggle
from google.colab import files
import os
#you will need to upload your own kaggle json file
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
dataset = 'ravidussilva/real-ai-art'
!kaggle datasets download -d $dataset

100% 9.95G/9.95G [01:51<00:00, 121MB/s]
100% 9.95G/9.95G [01:51<00:00, 96.0MB/s]


In [3]:
zip_file = "real-ai-art.zip"
destination_dir = f"/content/art"
if not os.path.exists(zip_file):
    print(f"Error: {zip_file} not found.")
else:
    !unzip -q $zip_file -d $destination_dir
    !rm $zip_file

In [4]:
!pip install tensorflow
!pip install tensorflow-hub

# Loading the Feature extraction VGG16 model

In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

#variable placeholders for image dimesnions
img_height = 256
img_width = 256

#drop top
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

#freeze layers
for layer in base_model.layers:
    layer.trainable = False

#feature extraction at last layer
output = base_model.output

#feature extraction model
feature_extraction_model = Model(inputs=base_model.input, outputs=output)

#model summary
print(feature_extraction_model.summary())


58889256/58889256 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                              

# Feature Extraction
Decision to extract features style by style as they are stored in folders

Create a function to test if the image is within the specified 256 x 256 pixels to normalize all inputs

In [6]:
from PIL import Image
import numpy as np

def preprocess_image(img_path):
    #target size of images256 x 256
    target_size = (256,256)

    img = Image.open(img_path)

    if img.size != target_size:
        img = img.resize(target_size, resample=Image.BICUBIC)

    #return the image array
    return (np.array(img))

Create a function to process all the images in a folder

In [14]:
import os
import time
import pandas as pd
from tensorflow.keras.applications.vgg16 import preprocess_input

def extract_features_from_folder(base,source,style):
    input_folder_path = base+source+style
    output_path = source+style
    feature_list = []
    i = 0
    for filename in os.listdir(input_folder_path):
      if i<10:
        print(i)
        img_path = os.path.join(input_folder_path, filename)
        if os.path.isfile(img_path):
          start_time = time.time()
          img_array = preprocess_image(img_path)
          img_array = np.expand_dims(img_array, axis=0)
          img_array = preprocess_input(img_array)

          features = feature_extraction_model.predict(img_array)
          feature_list.append(features.flatten())
          end_time = time.time()
        i+=1
      else:
        break

    #         end_time = time.time()
    # for filename in os.listdir(input_folder_path):
    #     img_path = os.path.join(input_folder_path, filename)
    #     if os.path.isfile(img_path):

    #         start_time = time.time()

    #         img_array = preprocess_image(img_path)
    #         img_array = np.expand_dims(img_array, axis=0)
    #         img_array = preprocess_input(img_array)

    #         features = feature_extraction_model.predict(img_array)
    #         feature_list.append(features.flatten())

    #         end_time = time.time()
            #print(f'{img_path} {end_time - start_time}')

    df = pd.DataFrame(feature_list)
    df.to_csv((output_path + '.csv'), index=False)

testing on a sample folder, remember to stop

In [15]:
# extract_features_from_folder('train/AI_LD_art_nouveau')

base = '/content/art/Real_AI_SD_LD_Dataset/train/'
source = 'AI_LD_'
style = 'art_nouveau'

extract_features_from_folder(base,source,style)

0
1/1 [==============================] - 1s 741ms/step
1
1/1 [==============================] - 1s 705ms/step
2
1/1 [==============================] - 1s 695ms/step
3
1/1 [==============================] - 1s 709ms/step
4
1/1 [==============================] - 1s 709ms/step
5
1/1 [==============================] - 1s 693ms/step
6
1/1 [==============================] - 1s 916ms/step
7
1/1 [==============================] - 1s 1s/step
8
1/1 [==============================] - 1s 1s/step
9
1/1 [==============================] - 1s 936ms/step
